# Design of Experiments module

In [1]:
import os.path as pth
import logging
import fastoad.api as oad

In [2]:
DATA_FOLDER = "data"

WORK_FOLDER = "workdir"

CONFIGURATION_FILE_NAME = pth.join(DATA_FOLDER, "beam_problem.yml")

logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

In [82]:
import pandas as pd
import contextlib
import os
import os.path as pth

def run_doe(
    doe_driver,
    x_dict: dict,
    configuration_file: str,
    nested_optimization: bool = False,
    doe_file_path: str = None
) -> pd.DataFrame:
    """
    Function for running Designs of Experiments (DoE) on FAST-OAD problems.
    If an optimization problem is declared in the configuration file,
    a nested optimization (sub-problem) is run (e.g. to ensure system optimality at each simulation).

    :param doe_driver: driver to be used for running the DoE
    :param x_dict: inputs dictionary {name: {'lower': lower_bound, 'upper': upper_bound}}
    :param configuration_file: configuration file for the problem
    :param nested_optimization: flag to run DoE on top of the optimization problem declared in the configuration file.
    If False, the DoE is run on the model only.
    :param doe_file_path: path for saving the results

    :return: dataframe of the design of experiments results
    """

    class SubProbComp(om.ExplicitComponent):
        """
        Sub-problem component for nested optimization.
        Inspired from https://github.com/OpenMDAO/RevHack2020/blob/master/solution_approaches/sub_problems.md
        """

        def initialize(self):
            self.options.declare("conf")
            self.options.declare("x_list")
            self.options.declare("y_list")

        def setup(self):
            # create a sub-problem to use later in the compute
            conf = self.options["conf"]
            prob = conf.get_problem(read_inputs=True)  # get conf file (design variables, objective, driver...)
            p = self._prob = prob
            p.setup()

            # define the i/o of the component
            x_list = self._x_list = self.options["x_list"]
            y_list = self._y_list = self.options["y_list"]

            for x in x_list:
                self.add_input(x)

            for y in y_list:
                self.add_output(y)

            # set counter and output variable for recording optimization failure or success
            self._fail_count = 0
            self.add_output('success')

            self.declare_partials("*", "*", method="fd")

        def compute(self, inputs, outputs):
            p = self._prob
            x_list = self._x_list
            y_list = self._y_list

            for x in x_list:
                p[x] = inputs[x]

            with open(os.devnull, "w") as f, contextlib.redirect_stdout(
                f
            ):  # turn off convergence messages
                fail = p.run_driver()

            for y in y_list:
                outputs[y] = p[y]

            if fail:
                self._fail_count += 1
            outputs['success'] = not fail

    # Get problem definition
    conf = oad.FASTOADProblemConfigurator(configuration_file)

    # Get inputs and outputs names
    x_list = list(x_dict.keys())
    prob = conf.get_problem(read_inputs=True)
    prob.setup()
    prob.final_setup()
    outputs = prob.model.get_io_metadata(
            "output", excludes="_auto_ivc.*"
        )
    indep_outputs = prob.model.get_io_metadata(
            "output",
            tags=["indep_var", "openmdao:indep_var"],
            excludes="_auto_ivc.*",
        )
    for abs_name, metadata in indep_outputs.items():
        del outputs[abs_name]
    y_list = [y['prom_name'] for y in outputs.values()]

    # Declare nested optimization if DoE must be run on top of the optimization problem
    if nested_optimization and conf.get_optimization_definition():
        prob = om.Problem()  # redefine DoE problem with optimization as a sub-problem
        prob.model.add_subsystem(
            "sub_prob",
            SubProbComp(
                conf=conf,
                x_list=x_list,
                y_list=y_list,
            ),
            promotes=["*"],
        )
        prob.setup()

    # Add input parameters for DoE
    for name, parameters in x_dict.items():
        prob.model.add_design_var(
            name, lower=parameters["lower"], upper=parameters["upper"]
        )

    # Setup driver
    prob.driver = doe_driver

    # Attach recorder to the driver
    if os.path.exists("cases.sql"):
        os.remove("cases.sql")
    prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))
    prob.driver.recording_options["includes"] = ["*"]

    # Run problem
    prob.setup()
    with open(os.devnull, "w") as f, contextlib.redirect_stdout(
            f
    ):  # turn off convergence messages
        prob.run_driver()
    prob.cleanup()

    # Get results from recorded cases
    df = pd.DataFrame()
    cr = om.CaseReader("cases.sql")
    cases = cr.list_cases("driver", out_stream=None)
    for case in cases:
        df_case = pd.DataFrame(cr.get_case(case).outputs)  # variables values for the case
        if not nested_optimization:
            df_case["success"] = cr.get_case(case).success  # success flag for the case
        df = pd.concat([df, df_case], ignore_index=True)
    os.remove("cases.sql")

    # Print number of failures
    fail_count = df["success"][df["success"] == 0].count()
    if fail_count > 0:
        print("%d out of %d cases failed. Check 'success' flag in DataFrame." % (fail_count, len(cases)))

    # save to .csv
    if not doe_file_path:
        doe_file_path = pth.join(pth.dirname(configuration_file), "doe.csv")
    doe_file_path = pth.abspath(doe_file_path)
    df.to_csv(doe_file_path)

    return df


In [83]:
#from fastoad.openmdao.doe import run_doe
import openmdao.api as om

driver = om.DOEDriver(om.UniformGenerator(num_samples=10))
x_dict = {"data:material:density": {"lower": 2000.0, "upper": 3000.0},
          "data:material:E": {"lower": 6e10, "upper": 8e10},
         }

In [90]:
df = run_doe(driver, x_dict, CONFIGURATION_FILE_NAME, nested_optimization=True)

INFO    : Loaded variable descriptions in D:\THESE\Tools\FAST-OAD-fork\FAST-OAD\src\fastoad\notebooks\03_Design_of_Experiments\data\../modules
INFO    : Loading bundles from D:\THESE\Tools\FAST-OAD-fork\FAST-OAD\src\fastoad\notebooks\03_Design_of_Experiments\data\../modules
INFO    : Installed bundle modules (ID 215 )
INFO    : Installed bundle modules.weight (ID 219 )
INFO    : Installed bundle modules.displacements (ID 216 )
INFO    : Installed bundle modules.section_properties (ID 217 )
INFO    : Installed bundle modules.stresses (ID 218 )


4 out of 10 cases failed. Check 'success' flag in DataFrame.


C:\Users\f.pollet.ISAE-SUPAERO\AppData\Roaming\Python\Python38\site-packages\openmdao\visualization\opt_report\opt_report.py:119: DriverWarning:The optimizer report is not applicable for Driver type 'DOEDriver', which does not support optimization


In [91]:
df

,data:material:density,data:material:E,data:geometry:Ixx,data:geometry:h,data:weight:linear_weight,success
0,2196.415546,7.060220e+10,0.000002,0.039404,254.712489,1.0
1,2591.944721,7.222808e+10,0.000001,0.038412,293.011111,0.0
2,2417.273746,7.591221e+10,0.000001,0.037847,269.245689,0.0
3,2478.517973,6.322862e+10,0.000002,0.039713,289.681267,0.0
4,2936.621898,6.272465e+10,0.000001,0.039011,337.152151,0.0
5,2231.517576,7.435531e+10,0.000001,0.037955,249.261461,1.0
6,2819.665014,7.297453e+10,0.000001,0.037379,310.178776,1.0
7,2136.683940,6.489521e+10,0.000002,0.039694,249.606436,1.0
8,2796.345544,6.438154e+10,0.000001,0.038876,319.936900,1.0
9,2022.851509,6.440651e+10,0.000002,0.039980,238.011134,1.0


In [92]:
df.describe()

,data:material:density,data:material:E,data:geometry:Ixx,data:geometry:h,data:weight:linear_weight,success
count,10.000000,1.000000e+01,1.000000e+01,10.000000,10.000000,10.000000
mean,2462.783747,6.857089e+10,1.465431e-06,0.038827,281.079741,0.600000
std,317.362237,5.113920e+09,1.007963e-07,0.000896,34.010203,0.516398
min,2022.851509,6.272465e+10,1.305651e-06,0.037379,238.011134,0.000000
25%,2205.191054,6.438778e+10,1.379337e-06,0.038069,250.882949,0.000000
50%,2447.895859,6.774871e+10,1.476560e-06,0.038944,279.463478,1.000000
75%,2745.245338,7.278792e+10,1.555125e-06,0.039622,305.886860,1.000000
max,2936.621898,7.591221e+10,1.597665e-06,0.039980,337.152151,1.000000


In [75]:
df["success"][df["success"] == 0].count()

0

In [72]:
5 in df["success"]

True